In [3]:
import pygame
import numpy as np
import heapq

# ------------------------------------------------------------
# ----------------------- PROPERTIES -------------------------
# ------------------------------------------------------------

# Configuraciones iniciales
ANCHO_VENTANA = 400
FILAS = 6
VENTANA = pygame.display.set_mode((ANCHO_VENTANA, ANCHO_VENTANA + 50))  # Espacio adicional para el boton
pygame.display.set_caption("Visualización de Nodos")

# Colores (RGB)
BLANCO = (255, 255, 255)
NEGRO = (0, 0, 0)
GRIS = (128, 128, 128)
VERDE = (0, 255, 0)
ROJO = (255, 0, 0)
NARANJA = (255, 165, 0)
PURPURA = (128, 0, 128)
AZUL = (0, 0, 255)

initial_position = {
    "x": None,
    "y": None,
}

final_position = {
    "x": None,
    "y": None,
}


# ------------------------------------------------------------
# ----------------------- CLASSES -------------------------
# ------------------------------------------------------------

class Nodo:
    def __init__(self, fila, col, ancho, total_filas):
        self.fila = fila
        self.col = col
        self.x = fila * ancho
        self.y = col * ancho
        self.color = BLANCO
        self.ancho = ancho
        self.total_filas = total_filas

    def get_pos(self):
        return self.fila, self.col

    def es_pared(self):
        return self.color == NEGRO

    def es_inicio(self):
        return self.color == NARANJA

    def es_fin(self):
        return self.color == PURPURA

    def restablecer(self):
        self.color = BLANCO

    def hacer_inicio(self):
        self.color = NARANJA

    def hacer_pared(self):
        self.color = NEGRO

    def hacer_fin(self):
        self.color = PURPURA
        
    def do_route(self):
        self.color = VERDE

    def dibujar(self, ventana):
        pygame.draw.rect(ventana, self.color, (self.x, self.y, self.ancho, self.ancho))

# ------------------------------------------------------------
# ----------------------- FUNCTIONS -------------------------
# ------------------------------------------------------------

def astar(matriz, inicio, fin):
    filas = len(matriz)
    columnas = len(matriz[0])

    # Función para calcular la distancia heurística (usaremos la distancia de Manhattan)
    def heuristica(a, b):
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    # Movimientos posibles (arriba, abajo, izquierda, derecha)
    movimientos = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    # Lista de nodos abiertos (prioridad por costo f)
    open_set = []
    heapq.heappush(open_set, (0, inicio))

    # Conjuntos para rastrear nodos explorados
    closed_set = set()

    # Diccionarios para almacenar el camino y costos
    came_from = {}
    g_score = {pos: float('inf') for fila in range(filas) for pos in [(fila, col) for col in range(columnas)]}
    g_score[inicio] = 0

    f_score = {pos: float('inf') for fila in range(filas) for pos in [(fila, col) for col in range(columnas)]}
    f_score[inicio] = heuristica(inicio, fin)

    while open_set:
        current = heapq.heappop(open_set)[1]

        if current == fin:
            # Reconstruir el camino
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(inicio)
            return path[::-1]  # Devolver el camino invertido

        closed_set.add(current)

        for movimiento in movimientos:
            neighbor = (current[0] + movimiento[0], current[1] + movimiento[1])

            # Verificar si el vecino está dentro de los límites
            if 0 <= neighbor[0] < filas and 0 <= neighbor[1] < columnas:
                # Verificar si el vecino no es una pared ('P')
                if matriz[neighbor[0]][neighbor[1]] in ['C', 'I', 'F']:
                    if neighbor in closed_set:
                        continue

                    tentative_g_score = g_score[current] + 1

                    if tentative_g_score < g_score[neighbor]:
                        came_from[neighbor] = current
                        g_score[neighbor] = tentative_g_score
                        f_score[neighbor] = tentative_g_score + heuristica(neighbor, fin)

                        # Añadir el vecino al open_set si no está ya en él
                        if not any(neighbor == item[1] for item in open_set):
                            heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return None  # No se encontró un camino



def crear_grid(filas, ancho):
    grid = []
    ancho_nodo = ancho // filas
    for i in range(filas):
        grid.append([])
        for j in range(filas):
            nodo = Nodo(i, j, ancho_nodo, filas)
            grid[i].append(nodo)
    return grid

def dibujar_grid(ventana, filas, ancho):
    ancho_nodo = ancho // filas
    for i in range(filas):
        pygame.draw.line(ventana, GRIS, (0, i * ancho_nodo), (ancho, i * ancho_nodo))
        for j in range(filas):
            pygame.draw.line(ventana, GRIS, (j * ancho_nodo, 0), (j * ancho_nodo, ancho))

def draw_start_button(ventana, ancho):
    fuente = pygame.font.SysFont(None, 40)
    texto = fuente.render("Start", True, BLANCO)
    boton_rect = pygame.Rect((ancho // 2) - 50, ancho + 10, 100, 40)
    pygame.draw.rect(ventana, AZUL, boton_rect)
    ventana.blit(texto, (boton_rect.x + 25, boton_rect.y + 5))
    return boton_rect

def dibujar(ventana, grid, filas, ancho):
    ventana.fill(BLANCO)
    for fila in grid:
        for nodo in fila:
            nodo.dibujar(ventana)

    dibujar_grid(ventana, filas, ancho)
    boton_rect = draw_start_button(ventana, ancho)

    pygame.display.update()
    return boton_rect

def obtener_click_pos(pos, filas, ancho):
    ancho_nodo = ancho // filas
    y, x = pos
    fila = y // ancho_nodo
    col = x // ancho_nodo
    return fila, col

def print_formated_grid(grid):
    for fila in grid:
        print([nodo.color for nodo in fila])
        
    current_grid = get_grid_formated(grid)
            
    print('---------------BY COLUMN-----------------')
    
    current_grid_rot90 = current_grid
    for column in current_grid_rot90:
        for cell in column:
            print(cell, end=" ") 
        print("")
        
    print('------------ LIKE A SCREEN --------------')
    current_grid_rot90 = np.transpose(current_grid)
    for column in current_grid_rot90:
        for cell in column:
            print(cell, end=" ") 
        print("")

def get_grid_formated(grid):
    
    current_grid = []
    for fila in grid:
        fila_colores = []
        for nodo in fila:
            if nodo.color == BLANCO:
                fila_colores.append('C')
            elif nodo.color == NEGRO:
                fila_colores.append('P')
            elif nodo.color == NARANJA:
                fila_colores.append('I')
            elif nodo.color == PURPURA:
                fila_colores.append('F')
        current_grid.append(fila_colores)
            
    return current_grid
    
def find_shortest_route(grid):
    if initial_position.get('x') != None and initial_position.get('y') != None and final_position.get('x') != None and final_position.get('y'):
        initial_position_x_y = (initial_position.get('y'), initial_position.get('x'))
        final_position_x_y = (final_position.get('y'), final_position.get('x'))
        
        print(f'initial_position x:{initial_position_x_y[1]}')
        print(f'initial_position y:{initial_position_x_y[0]}')
        print(f'final_position x:{final_position_x_y[1]}')
        print(f'final_position y:{final_position_x_y[0]}')
        
        print('-------------------------------------')
        current_formated_grid = get_grid_formated(grid)
        for column in current_formated_grid:
            for cell in column:
                print(cell, end=" ") 
            print("")
        
        camino = astar(current_formated_grid, initial_position_x_y, final_position_x_y)

        if camino:
            print("Camino encontrado:")
            for paso in camino:
                print(paso)
        else:
            print("No se encontró un camino.")


        # Visualización del camino en la matriz
        matriz = current_formated_grid
        
        if camino:
            # Marcar el camino en la matriz
            for paso in camino:
                fila, columna = paso
                if matriz[fila][columna] not in ['I', 'F']:
                    matriz[fila][columna] = '*'

            # Imprimir la matriz resultante
            print("\nMatriz con el camino encontrado:")
            for fila in matriz:
                print(' '.join(fila))
                
        #---------------------------
        if camino:
            return camino
        else:
            print("No se encontró un camino.")
    
    return       

def draw_find_shortest_route(grid, shortest_route):
    if shortest_route:
        print(initial_position)
        print(final_position)
        print("shortest_route")
        for step in shortest_route:
            node = grid[step[0]][step[1]]
            row, col = node.get_pos()
            print('[prev]''r' + str(row) + 'c' + str(col))
            
            if((row == initial_position.get('y') and col == initial_position.get('x')) or (row == final_position.get('y') and col == final_position.get('x'))):
                continue
            print('r' + str(row) + 'c' + str(col))
            node.do_route()
    else:
        print('No grid or no shortest_route in: draw_find_shortest_route')
        return
    
    print('---------------------------------------------')
    print(grid)
    
    return
# ------------------------------------------------------------
# ----------------------- MAIN PROCESS -------------------------
# ------------------------------------------------------------

def main(ventana, filas, ancho):
    pygame.init()
    pygame.font.init()
    grid = crear_grid(filas, ancho)

    inicio = None
    fin = None

    corriendo = True

    while corriendo:
        boton_rect = dibujar(ventana, grid, filas, ancho)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                corriendo = False

            if pygame.mouse.get_pressed()[0]:  # Click izquierdo
                pos = pygame.mouse.get_pos()
                if pos[1] > ancho:  # Verifica si el click es en el boton
                    if boton_rect.collidepoint(pos):
                        print("--------- Start button presed ---------")
                        shortest_route = find_shortest_route(grid)
                        draw_find_shortest_route(grid, shortest_route)
                        
                else:
                    fila, col = obtener_click_pos(pos, filas, ancho)
                    nodo = grid[fila][col]
                    if not inicio and nodo != fin:
                        inicio = nodo
                        inicio.hacer_inicio()
                        
                        # print('fila:' + str(fila))
                        # print('col:' + str(col))
                        # print_formated_grid(grid)
                        
                        initial_position["x"] = col
                        initial_position["y"] = fila

                    elif not fin and nodo != inicio:
                        fin = nodo
                        fin.hacer_fin()
                        final_position["x"] = col
                        final_position["y"] = fila

                    elif nodo != fin and nodo != inicio:
                        nodo.hacer_pared()

            elif pygame.mouse.get_pressed()[2]:  # Click derecho
                pos = pygame.mouse.get_pos()
                if pos[1] <= ancho:  # Solo permite restablecer nodos dentro del grid
                    fila, col = obtener_click_pos(pos, filas, ancho)
                    nodo = grid[fila][col]
                    nodo.restablecer()
                    if nodo == inicio:
                        inicio = None
                    elif nodo == fin:
                        fin = None

    pygame.quit()

main(VENTANA, FILAS, ANCHO_VENTANA)

--------- Start button presed ---------
initial_position x:4
initial_position y:0
final_position x:1
final_position y:4
-------------------------------------
C P C C I C 
C P C P P C 
C C C C P C 
C P P C C C 
C F C P P C 
C C P C C C 
Camino encontrado:
(0, 4)
(0, 3)
(0, 2)
(1, 2)
(2, 2)
(2, 1)
(2, 0)
(3, 0)
(4, 0)
(4, 1)

Matriz con el camino encontrado:
C P * * I C
C P * P P C
* * * C P C
* P P C C C
* F C P P C
C C P C C C
{'x': 4, 'y': 0}
{'x': 1, 'y': 4}
shortest_route
[prev]r0c4
[prev]r0c3
r0c3
[prev]r0c2
r0c2
[prev]r1c2
r1c2
[prev]r2c2
r2c2
[prev]r2c1
r2c1
[prev]r2c0
r2c0
[prev]r3c0
r3c0
[prev]r4c0
r4c0
[prev]r4c1
---------------------------------------------
[[<__main__.Nodo object at 0x000001B2EEB9A480>, <__main__.Nodo object at 0x000001B2FF265CA0>, <__main__.Nodo object at 0x000001B2FF265D90>, <__main__.Nodo object at 0x000001B2FF265DC0>, <__main__.Nodo object at 0x000001B2FF265DF0>, <__main__.Nodo object at 0x000001B2FF265E20>], [<__main__.Nodo object at 0x000001B2FF265E50>